In [12]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from cpmm import CPMM, run_experiment

# Slippage intuition
Slippage is an inefficiency of CPMM that results in slightly (or significantly) worse odds when betting. For example in perfect market with outcome probabilities split equally, if you stake \$100 for "yes" and "yes" wins then you receive \$200 (\$100 stake + \$100 profit). In case of CPMM with \$1000 liquidity you will receive ~\$191 payout and you lose \$9 to the market inefficiency. In other words your odds are not 1:1 but ~0.91:1. This difference of \$9 is called **slippage**. In our simulations we define slippage as  

slippage = (actual payout - ideal payout) / actual payout

Slippage has negative effect on trading.

1. Slippage prevents large bets when liquidity is low. You will not bet \$1000 if you lose \$100 to inefficiencies
2. Slippage impacts much more markets that have large difference of odds. 1:1 is the best case. 9:1 has large slippage (inefficiency) when buying yes of the option


> working questions
> 1. how much liquidity we need to reduce slippage to realistic values? ie. 1 or 0.5% 0.5 is good slippage on uniswap.
> 2. visualise loss of the token due to slippage due to marginal price
> 3. slippage vs. initial liquidity distribution vs. betters prios. ie 90:10. is slippage the same?

In [13]:
# cpmm = CPMM()
# cpmm.create_event(1000, initial_yes_to_no=1)
# print(cpmm.lp_yes)
# print(cpmm.lp_no)
# print(cpmm.calc_marginal_price(1))
# print(cpmm.calc_marginal_price(0))
# print(cpmm.calc_slippage(1, 100))
# print(cpmm.calc_slippage(0, 100))
# print(cpmm.calc_buy(1, 100))
# print(cpmm.calc_buy(0, 100))
# print(cpmm.buy_token(1, 100))
# print(cpmm.lp_yes)
# print(cpmm.lp_no)
# print(cpmm.calc_marginal_price(1))
# print(cpmm.calc_marginal_price(0))
# print(cpmm.calc_slippage(1, 100))
# print(cpmm.buy_token(1, 100))

# Slippage is a function of bet amount / liquidity amount
Slippage is function of bet amount / liquidity amount. For example we have the same slippage for bet amount 100 with liquidity amount 1000 and bet amount 1000 with liquidity 10000.
**Example above assumes 1:1 yes:no odds**

In [14]:
cpmm = CPMM()
initial_liquidity = 1000
cpmm.create_event(initial_liquidity)
# slippage vs increased bets
slippages = [[amount / initial_liquidity , cpmm.calc_slippage(0, amount)] for amount in range(1, 3000, 5)]
df = pd.DataFrame(data=slippages, columns=["bet amount/liquidity", "slippage"])

# layout = px.Layout(yaxis=dict(tickformat=".2%"))
fig = px.line(df, x="bet amount/liquidity", y="slippage", title='Slippage vs bet amount as % of liquidity')
fig.show()


invariant P 1 1.0


# Zoom in at realistic ranges of slippage (up to 2%).
We have 1% of slippage when bet is 2% of liquidity and 2% of slippage when bet is 4.18% of liquidity. If we assume that 1% of slippage is realistic maximum we need \$1000 liquidity to have \$20 bets or \$50,000 liquidity to have \$1000 bets. With 1% slippage trader loses \$10 out of \$1000 bet

In [15]:
# slippage vs increased bets
cpmm = CPMM()
initial_liquidity = 1000
cpmm.create_event(initial_liquidity)
zoom = 20.0
slippages = [[(amount / zoom) / initial_liquidity , cpmm.calc_slippage(0, amount / zoom)] for amount in range(1, 1000, 5)]
df = pd.DataFrame(data=slippages, columns=["bet amount/liquidity", "slippage"])
fig = px.line(df, x="bet amount/liquidity", y="slippage", title='Slippage vs bet amount as % of liquidity')
fig.show()

invariant P 1 1.0


# option with higher odds (higher reward) has bigger slippage
With odds 1:1 we have 0.5% slippage for both outcomes, with yes:no odds 1:8 we have 3.5% slippage for NO which is quite large (and very small slippage for YES). However NO option is much more interesting to gamblers as possible payout is higher

In [16]:
from IPython.display import Markdown, display

cpmm = CPMM()
initial_liquidity = 1000
amount = 10
cpmm.create_event(initial_liquidity)

slippages = []
odds_yes = 1
odds_no = 1
for i in range(0, 300):
    # (initial_liquidity + amount*i) / initial_liquidity
    slippages.append([odds_yes, odds_no, cpmm.calc_slippage(1, amount), cpmm.calc_slippage(0, amount)])
    yes_reward = cpmm.buy_token(1, amount)[1]
    no_reward = cpmm.calc_buy(0, amount)[0]
    odds_yes = yes_reward / 2 / amount
    odds_no = no_reward / 2 / amount

df = pd.DataFrame(data=slippages, columns=["odds yes", "odds no", "slippage yes", "slippage_no"])
fig = px.line(df, x="odds yes", y=["slippage yes", "slippage_no"], title='slippage vs yes odds')
display(fig.show())
fig = px.line(df, x="odds no", y=["slippage yes", "slippage_no"], title='slippage vs no odds')
display(fig.show())
display(df)

invariant P 1 1.0


None

None

,odds yes,odds no,slippage yes,slippage_no
0,1.000000,1.000000,0.004950,0.004950
1,0.995050,1.004950,0.004853,0.005049
2,0.985343,1.014948,0.004758,0.005148
3,0.975919,1.025042,0.004666,0.005248
4,0.966766,1.035234,0.004575,0.005348
...,...,...,...,...
295,0.532127,8.004810,0.000152,0.035710
296,0.531965,8.042132,0.000151,0.035808
297,0.531804,8.079542,0.000150,0.035906
298,0.531644,8.117042,0.000149,0.036004


# mitigation via increasing liquidity. any consequences?
When increasing liquidity

1. Slippage is going down and rewards are going up. But at the end who's paying for this? Liquidity Provider with permanent loss!
2. Liquidity adds inertia to the market. Look how much slower odds are changing due to new bets. More liquidity may slow reactions to new information and basically require that LPs remove liquidity to let market rebalance

Examples below show 10x liquidity increase

In [19]:
cpmm = CPMM()
initial_liquidity = 1000
amount = 10
cpmm.create_event(initial_liquidity)

# print(cpmm.buy_token(1, amount))
# print(cpmm.calc_payout())
# print(min(cpmm.lp_yes, cpmm.lp_no))
# print(cpmm.calc_outstanding_token())
# print((cpmm.lp_yes/cpmm.lp_no))

vals = []
yes_total_reward = 0
for i in range(1, 100):
    slippage = cpmm.calc_slippage(1, amount)
    yes_reward = cpmm.buy_token(1, amount)[1]
    yes_total_reward += yes_reward
    vals.append([yes_reward, yes_reward / 2 / amount, i * amount, yes_total_reward, (yes_total_reward / 2) / (i * amount), slippage, cpmm.calc_payout(),   
    cpmm.calc_impermanent_loss(),
    cpmm.calc_outstanding_token()[1]])

df = pd.DataFrame(data=vals, columns=["yes reward", "yes odds", "yes stake", "yes total reward", "total odds", "slippage", "total reward", "impermanent loss", "outstanding token"])
fig = px.line(df, x="yes odds", y=["impermanent loss", "yes total reward"], title='impermanent loss vs change from intial yes odds')
display(fig.show())
display(df)

invariant P 1 1.0


None

,yes reward,yes odds,yes stake,yes total reward,total odds,slippage,total reward,impermanent loss,outstanding token
0,19.900990,0.995050,10,19.900990,0.995050,0.004950,19.900990,9.900990,19.900990
1,19.706853,0.985343,20,39.607843,0.990196,0.004853,39.607843,19.607843,39.607843
2,19.518370,0.975919,30,59.126214,0.985437,0.004758,59.126214,29.126214,59.126214
3,19.335325,0.966766,40,78.461538,0.980769,0.004666,78.461538,38.461538,78.461538
4,19.157509,0.957875,50,97.619048,0.976190,0.004575,97.619048,47.619048,97.619048
...,...,...,...,...,...,...,...,...,...
94,12.643405,0.632170,950,1437.179487,0.756410,0.001077,1437.179487,487.179487,1437.179487
95,12.616431,0.630822,960,1449.795918,0.755102,0.001062,1449.795918,489.795918,1449.795918
96,12.589868,0.629493,970,1462.385787,0.753807,0.001048,1462.385787,492.385787,1462.385787
97,12.563708,0.628185,980,1474.949495,0.752525,0.001035,1474.949495,494.949495,1474.949495


In [20]:
cpmm = CPMM()
initial_liquidity = 10000
amount = 10
cpmm.create_event(initial_liquidity)

# print(cpmm.buy_token(1, amount))
# print(cpmm.calc_payout())
# print(min(cpmm.lp_yes, cpmm.lp_no))
# print(cpmm.calc_outstanding_token())
# print((cpmm.lp_yes/cpmm.lp_no))

vals = []
yes_total_reward = 0
for i in range(1, 100):
    slippage = cpmm.calc_slippage(1, amount)
    yes_reward = cpmm.buy_token(1, amount)[1]
    yes_total_reward += yes_reward
    vals.append([yes_reward, yes_reward / 2 / amount, i * amount, yes_total_reward, (yes_total_reward / 2) / (i * amount), slippage, cpmm.calc_payout(),   
    cpmm.calc_impermanent_loss(),
    cpmm.calc_outstanding_token()[1]])

df = pd.DataFrame(data=vals, columns=["yes reward", "yes odds", "yes stake", "yes total reward", "total odds", "slippage", "total reward", "impermanent loss", "outstanding token"])
fig = px.line(df, x="yes odds", y=["impermanent loss", "yes total reward"], title='impermanent loss vs change from intial yes odds')
display(fig.show())
display(df)

invariant P 1 1.0


None

,yes reward,yes odds,yes stake,yes total reward,total odds,slippage,total reward,impermanent loss,outstanding token
0,19.990010,0.999500,10,19.990010,0.999500,0.000500,19.990010,9.990010,19.990010
1,19.970070,0.998503,20,39.960080,0.999002,0.000499,39.960080,19.960080,39.960080
2,19.950189,0.997509,30,59.910269,0.998504,0.000498,59.910269,29.910269,59.910269
3,19.930368,0.996518,40,79.840637,0.998008,0.000497,79.840637,39.840637,79.840637
4,19.910606,0.995530,50,99.751244,0.997512,0.000496,99.751244,49.751244,99.751244
...,...,...,...,...,...,...,...,...,...
94,18.347733,0.917387,950,1817.579909,0.956621,0.000416,1817.579909,867.579909,1817.579909
95,18.332500,0.916625,960,1835.912409,0.956204,0.000415,1835.912409,875.912409,1835.912409
96,18.317309,0.915865,970,1854.229717,0.955789,0.000414,1854.229717,884.229717,1854.229717
97,18.302159,0.915108,980,1872.531876,0.955373,0.000413,1872.531876,892.531876,1872.531876
